In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


# Transfer learning experiments

This notebook includes:
1) Implement a transfer learning from ImageNet > 13 diseases > 1 disease. 
2) Compared performance against ImageNet > 1 disease and ImageNet > MURA > 1 disease 

Results for disease corresponding to index 10.

## Imports & global variables

We choose `idx=10` because from previous experiments we have detected a strong signal and a sufficient amount of positive cases. 

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys; sys.path.append("..")
import warnings; warnings.filterwarnings('ignore')

from core import * 
from data_manipulation import Transform, RandomRotation, Flip, RandomCrop, multi_label_2_binary, balance_obs, DataBatches
from utils import save_model, load_model, lr_loss_plot, resize
from architectures import DenseNet121
from train_functions import OptimizerWrapper, TrainingPolicy, FinderPolicy, validate_multilabel, lr_finder, validate_binary, TTA_binary

seed = 42
r_pix = 8
BATCH_SIZE = 16
EPOCHS = 10
TRANSFORMATIONS = [RandomRotation(arc_width=20), Flip(), RandomCrop(r_pix=r_pix)]
PRETRAINED = True
NORMALIZE = True # ImageNet
FREEZE = True
GRADUAL_UNFREEZING = True
DATA = '14diseases'
N_SAMPLES = 1100


BASE_PATH = Path('../..')
PATH = BASE_PATH/'data'
SAVE_RESULTS = './results/'
SAVE_MODELS = './models'
IMG_FOLDER = PATH/'ChestXRay-250'

DISEASE = 'Emphysema'
tgt2idx = {disease: i for i, disease in enumerate([ 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia',
               'Pneumothorax', 'Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Pleural_Thickening', 'Hernia'])}

# Downstream task - 1 label (small data)

Balance training and testing.

In [3]:
def train(n_epochs, train_dl, valid_dl, model, max_lr=.01, wd=0, alpha=1./ 3,
          save_path=None, unfreeze_during_loop:tuple=None):
    
    if unfreeze_during_loop:
        total_iter = n_epochs*len(train_dl)
        first_unfreeze = int(total_iter*unfreeze_during_loop[0])
        second_unfreeze = int(total_iter*unfreeze_during_loop[1])

    best_loss = np.inf
    cnt = 0
    
    policy = TrainingPolicy(n_epochs=n_epochs, dl=train_dl, max_lr=max_lr)
    optimizer = OptimizerWrapper(model, policy, wd=wd, alpha=alpha)

    for epoch in tqdm_notebook(range(n_epochs), ):
        model.train()
        agg_div = 0
        agg_loss = 0
        train_dl.set_random_choices()
        for x, y in tqdm_notebook(train_dl, leave=False):

            if unfreeze_during_loop:
                if cnt == first_unfreeze: model.unfreeze(1)
                if cnt == second_unfreeze: model.unfreeze(0)

            out = model(x)
            loss = F.binary_cross_entropy_with_logits(input=out.squeeze(), target=y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            batch = y.shape[0]
            agg_loss += batch*loss.item()
            agg_div += batch
            cnt += 1


        val_loss, measure, _ = validate_binary(model, valid_dl)
        print(f'Ep. {epoch+1} - train loss {agg_loss/agg_div:.4f} -  val loss {val_loss:.4f} AUC {measure:.4f}')

        if save_path and val_loss < best_loss:
            save_model(model, save_path)
            best_loss = val_loss

# Comparison on the Downstream task

In [4]:
train_df = pd.read_csv(PATH/'train_df.csv')
valid_df = pd.read_csv(PATH/"val_df.csv")
test_df = pd.read_csv(PATH/"test_df.csv")

train_df = multi_label_2_binary(train_df, tgt2idx[DISEASE])
train_df = train_df.sample(frac=1, random_state=42)

valid_df = multi_label_2_binary(valid_df, tgt2idx[DISEASE])
valid_df = balance_obs(valid_df, amt=None, rate_positive=.5)

test_df = multi_label_2_binary(test_df, tgt2idx[DISEASE])
test_df = balance_obs(test_df, amt=None, rate_positive=.5)

In [5]:
# len(sample_train_df),len(valid_df),len(test_df)

## Without Mixup

In [6]:
p_positive = .20
N = int(p_positive*100)
sample_train_df = balance_obs(train_df, amt=50, rate_positive=p_positive)

In [7]:

train_dl = DataBatches(df=sample_train_df, transforms=TRANSFORMATIONS, shuffle=True,
                       img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                       r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

valid_dl = DataBatches(df=valid_df, transforms=None, shuffle=False,
                       img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                       r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

test_dl = DataBatches(df=test_df, transforms=TRANSFORMATIONS, shuffle=False, 
                      img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                      r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

### ImageNet

In [8]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained=True, freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_imagenet.pth'
epochs = 4
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.7418 -  val loss 0.7579 AUC 0.6201


Ep. 2 - train loss 0.4035 -  val loss 1.2777 AUC 0.6976


Ep. 3 - train loss 0.2048 -  val loss 1.5233 AUC 0.7246


Ep. 4 - train loss 0.1758 -  val loss 1.3686 AUC 0.7237

TTA loss 1.2922  auc 0.5895  accuracy 0.5046


(1.2922049251479615, 0.5895246218763838, 0.5045745654162854)

### MURA

In [9]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='MURA', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_MURA.pth'
epochs = 4
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.9020 -  val loss 0.6876 AUC 0.5550


Ep. 2 - train loss 0.4982 -  val loss 0.7755 AUC 0.7288


Ep. 3 - train loss 0.2886 -  val loss 0.8397 AUC 0.7510


Ep. 4 - train loss 0.2456 -  val loss 0.8181 AUC 0.7532

TTA loss 0.9312  auc 0.5746  accuracy 0.5018


(0.9311885021674666, 0.5745511861643042, 0.5018298261665142)

### CheXpert

In [22]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='chexpert', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_chexpert.pth'
epochs = 12
train(epochs, train_dl, valid_dl, model, 
      max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.7139 -  val loss 0.7311 AUC 0.3505


Ep. 2 - train loss 0.6965 -  val loss 0.7031 AUC 0.3867


Ep. 3 - train loss 0.6474 -  val loss 0.6947 AUC 0.5823


Ep. 4 - train loss 0.5742 -  val loss 0.7066 AUC 0.6471


Ep. 5 - train loss 0.5239 -  val loss 0.7306 AUC 0.6709


...

Ep. 9 - train loss 0.4328 -  val loss 0.8224 AUC 0.7051


Ep. 10 - train loss 0.4303 -  val loss 0.8322 AUC 0.7060


Ep. 11 - train loss 0.4315 -  val loss 0.8334 AUC 0.7087


Ep. 12 - train loss 0.4349 -  val loss 0.8340 AUC 0.7108

TTA loss 0.8479  auc 0.5329  accuracy 0.5000


(0.8478797607785066, 0.5329456601897293, 0.5)

In [23]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='chexpert', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_chexpert.pth'
epochs = 12
train(epochs, train_dl, valid_dl, model, 
      max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.6514 -  val loss 0.6800 AUC 0.6280


Ep. 2 - train loss 0.6371 -  val loss 0.6811 AUC 0.6337


Ep. 3 - train loss 0.5993 -  val loss 0.6894 AUC 0.6573


Ep. 4 - train loss 0.5558 -  val loss 0.7123 AUC 0.6790


Ep. 5 - train loss 0.5017 -  val loss 0.7290 AUC 0.7011


...

Ep. 9 - train loss 0.4336 -  val loss 0.7864 AUC 0.7318


Ep. 10 - train loss 0.4530 -  val loss 0.8305 AUC 0.7276


Ep. 11 - train loss 0.4339 -  val loss 0.7877 AUC 0.7305


Ep. 12 - train loss 0.4198 -  val loss 0.7844 AUC 0.7308

TTA loss 0.8042  auc 0.5152  accuracy 0.5000


(0.8042148654055181, 0.5151969323207067, 0.5)

### 13 diseases

In [ ]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='13diseases', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_13diseases.pth'
epochs = 7
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.6194 -  val loss 0.6447 AUC 0.7953


## With Mixup

In [ ]:
train_dl = DataBatches(df=sample_train_df, transforms=TRANSFORMATIONS, shuffle=True,
                       img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                       r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=True)

valid_dl = DataBatches(df=valid_df, transforms=None, shuffle=False,
                       img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                       r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

test_dl = DataBatches(df=test_df, transforms=TRANSFORMATIONS, shuffle=False, 
                      img_folder_path=IMG_FOLDER, batch_size=BATCH_SIZE, data='Pneumonia',
                      r_pix=r_pix, normalize=NORMALIZE, seed=seed, mixup=False)

### ImageNet

In [13]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained=True, freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_imagenet_mixup.pth'
epochs = 4
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

Ep. 1 - train loss 0.7317 -  val loss 0.7078 AUC 0.5944


Ep. 2 - train loss 0.5284 -  val loss 0.9939 AUC 0.7100


Ep. 3 - train loss 0.4013 -  val loss 1.4494 AUC 0.7533


Ep. 4 - train loss 0.3577 -  val loss 1.6444 AUC 0.7730


Ep. 5 - train loss 0.4012 -  val loss 1.8009 AUC 0.7639


Ep. 6 - train loss 0.2932 -  val loss 1.3900 AUC 0.7731



KeyboardInterrupt: 

### MURA

In [ ]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='MURA', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_MURA_mixup.pth'
epochs = 6
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

### CheXpert

In [ ]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='chexpert', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_chexpert_mixup.pth'
epochs = 6
train(epochs, train_dl, valid_dl, model, max_lr=5e-4, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

### 13 diseases

In [ ]:
drop_rate = 0 # Not good for CNN
model = DenseNet121(out_size=1, pretrained='13diseases', freeze=True, drop_rate=drop_rate).cuda()
model_p = f'./models/best_{N}_emphysema_13diseases_mixup.pth'
epochs = 5
train(epochs, train_dl, valid_dl, model, max_lr=1e-3, save_path=None, 
      unfreeze_during_loop = (.1,.2))
TTA_binary(model, test_dl)

# Results


| Method | test AUC   | test Accuracy (tr = .5) |
|------|------|-------| 
|   ImageNet + Mixup  | 0.611 | 0.552|
|   ImageNet  | 0.625 | 0.624|
|   MURA + Mixup  | 0.659| 0.615|
|   MURA  | 0.674 | 0.606|
|   CheXpert + Mixup  | 0.748| 0.513|
|   CheXpert  | 0.769 | 0.594 |

Mixup seems to don't help in the case of X-rays. 